In [3]:
import pandas as pd
import numpy as np
import json
from datetime import date
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [9]:
artist_df = pd.read_csv('./data/artists.csv')
tracks_df = pd.read_csv('./data/tracks.csv')
with open("./data/dict_artists.json", encoding='utf-8', errors='ignore') as json_data:
     data = json.load(json_data, strict=False)

In [10]:
artist_df.drop(['id'], axis=1, inplace=True)

In [15]:
artist_df = artist_df[artist_df['genres'] != '[]']
artist_df = artist_df.reset_index(drop=True)

In [ ]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(ds['description'])